In [ ]:
!pip install datasets transformers

     |████████████████████████████████| 290 kB 5.4 MB/s 
     |████████████████████████████████| 3.1 MB 41.4 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
     |████████████████████████████████| 1.1 MB 23.9 MB/s 
     |████████████████████████████████| 243 kB 48.8 MB/s 
     |████████████████████████████████| 132 kB 49.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.0 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 
     |████████████████████████████████| 192 kB 45.3 MB/s 
     |████████████████████████████████| 160 kB 50.9 MB/s 
     |████████████████████████████████| 271 kB 47.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%%capture

#import
from datasets import load_dataset, load_metric
from tqdm.auto import tqdm
import collections
import numpy as np
import transformers
from transformers import AutoTokenizer
from transformers import default_data_collator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

In [ ]:
%%capture
batch_size = 32
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
%%capture
# Download part of dataset
datasets = load_dataset("squad_v2")
part_datasets = load_dataset("squad_v2", split=['train[:6%]', 'validation[:10%]'])
datasets['train'] = part_datasets[0]
datasets['validation'] = part_datasets[1]

In [ ]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 7819
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1187
    })
})


In [ ]:
%%capture
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_features(examples, tokenizer, train=True):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    if train:
      # The offset mappings will give us a map from token to character position in the original context. This will
      # help us compute the start_positions and end_positions.
      offset_mapping = tokenized_examples.pop("offset_mapping")

      # Let's label those examples!
      tokenized_examples["start_positions"] = []
      tokenized_examples["end_positions"] = []

      for i, offsets in enumerate(offset_mapping):
          # We will label impossible answers with the index of the CLS token.
          input_ids = tokenized_examples["input_ids"][i]
          cls_index = input_ids.index(tokenizer.cls_token_id)

          # Grab the sequence corresponding to that example (to know what is the context and what is the question).
          sequence_ids = tokenized_examples.sequence_ids(i)

          # One example can give several spans, this is the index of the example containing this span of text.
          sample_index = sample_mapping[i]
          answers = examples["answers"][sample_index]
          # If no answers are given, set the cls_index as answer.
          if len(answers["answer_start"]) == 0:
              tokenized_examples["start_positions"].append(cls_index)
              tokenized_examples["end_positions"].append(cls_index)
          else:
              # Start/end character index of the answer in the text.
              start_char = answers["answer_start"][0]
              end_char = start_char + len(answers["text"][0])

              # Start token index of the current span in the text.
              token_start_index = 0
              while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                  token_start_index += 1

              # End token index of the current span in the text.
              token_end_index = len(input_ids) - 1
              while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                  token_end_index -= 1

              # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
              if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                  tokenized_examples["start_positions"].append(cls_index)
                  tokenized_examples["end_positions"].append(cls_index)
              else:
                  # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                  # Note: we could go after the last offset if the answer is the last word (edge case).
                  while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                      token_start_index += 1
                  tokenized_examples["start_positions"].append(token_start_index - 1)
                  while offsets[token_end_index][1] >= end_char:
                      token_end_index -= 1
                  tokenized_examples["end_positions"].append(token_end_index + 1)
    else:

      # We keep the example_id that gave us this feature and we will store the offset mappings.
      tokenized_examples["example_id"] = []

      for i in range(len(tokenized_examples["input_ids"])):
          # Grab the sequence corresponding to that example (to know what is the context and what is the question).
          sequence_ids = tokenized_examples.sequence_ids(i)
          context_index = 1 if pad_on_right else 0

          # One example can give several spans, this is the index of the example containing this span of text.
          sample_index = sample_mapping[i]
          tokenized_examples["example_id"].append(examples["id"][sample_index])

          # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
          # position is part of the context or not.
          tokenized_examples["offset_mapping"][i] = [
              (o if sequence_ids[k] == context_index else None)
              for k, o in enumerate(tokenized_examples["offset_mapping"][i])
          ]

    return tokenized_examples



In [ ]:
%%capture
tokenized_datasets = datasets.map(
    lambda x: prepare_features(x, tokenizer, train=True),
    batched=True,
    remove_columns=datasets["train"].column_names
)
validation_features = datasets["validation"].map(
    lambda x: prepare_features(x, tokenizer, train=False),
    batched=True,
    remove_columns=datasets["validation"].column_names
)

In [ ]:
def train_new_model():
  '''
  train a new model with his tokenizer
  '''
  model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
  model_name = model_checkpoint.split("/")[-1]
  args = TrainingArguments(
  f"{model_name}-finetuned-squad",
  evaluation_strategy = "epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  num_train_epochs=3,
  weight_decay=0.01,
  )
  data_collator = default_data_collator
  trainer = Trainer(
      model,
      args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["validation"],
      data_collator=data_collator,
      tokenizer=tokenizer,
  )
  trainer.train()
  return trainer

In [ ]:
trainer = train_new_model()

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,No log,2.722083
2,No log,2.502331
3,2.618200,2.613953


In [ ]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions


In [ ]:
def eval(model : transformers.models.distilbert.modeling_distilbert.DistilBertForQuestionAnswering, 
         tokenizer : transformers.models.distilbert.tokenization_distilbert_fast.DistilBertTokenizerFast):
  '''
  Eval a model with his tokenizer to get the squadv2 metrics
  '''
  trainer = Trainer(
      model,
      tokenizer=tokenizer,
  )

  validation_features_mv = datasets["validation"].map(
    lambda x: prepare_features(x, tokenizer, train=False),
    batched=True,
    remove_columns=datasets["validation"].column_names
  )
  raw_predictions = trainer.predict(validation_features_mv)
  validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

  final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)
  metric = load_metric("squad_v2")
  formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
  references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
  metrics = metric.compute(predictions=formatted_predictions, references=references)
  for result in metrics:
    print(f'{result} : {metrics[result]}')

  return formatted_predictions, references

In [ ]:
formatted_predictions, references = eval(trainer.model, trainer.tokenizer)

  0%|          | 0/2 [00:00<?, ?ba/s]

Post-processing 1187 example predictions split into 1194 features.


  0%|          | 0/1187 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

exact : 32.43470935130581
f1 : 36.58299276614811
total : 1187
HasAns_exact : 50.671140939597315
HasAns_f1 : 58.93290673392249
HasAns_total : 596
NoAns_exact : 14.043993231810491
NoAns_f1 : 14.043993231810491
NoAns_total : 591
best_exact : 50.12636899747262
best_exact_thresh : 0.0
best_f1 : 50.12636899747262
best_f1_thresh : 0.0


In [ ]:
%%capture

#dowload pretrained model from "mvonwyl/distilbert-base-uncased-finetuned-squad2"
tokenizer_mv = AutoTokenizer.from_pretrained("mvonwyl/distilbert-base-uncased-finetuned-squad2")
model_mv = AutoModelForQuestionAnswering.from_pretrained("mvonwyl/distilbert-base-uncased-finetuned-squad2")
trainer_mv = Trainer(
      model_mv,
      tokenizer=tokenizer_mv,
  )

In [ ]:
formatted_predictions, references = eval(model_mv, tokenizer_mv)

  0%|          | 0/2 [00:00<?, ?ba/s]

Post-processing 1187 example predictions split into 1194 features.


  0%|          | 0/1187 [00:00<?, ?it/s]

exact : 65.37489469250211
f1 : 69.09292293067756
total : 1187
HasAns_exact : 65.60402684563758
HasAns_f1 : 73.00889180992338
HasAns_total : 596
NoAns_exact : 65.14382402707275
NoAns_f1 : 65.14382402707275
NoAns_total : 591
best_exact : 65.54338668913226
best_exact_thresh : 0.0
best_f1 : 69.26141492730767
best_f1_thresh : 0.0


In [ ]:
# Get first ten incorrect prediction
nb = 0
for i, reference in enumerate(references[:200]):
  answers = reference['answers']['text']
  prediction_text = formatted_predictions[i]['prediction_text']
  if prediction_text not in answers and answers != []:
    print("Index : ", i)
    print("Correct answers : ", answers)
    print("Predictions : ", prediction_text)
    print()
    nb += 1
  if nb == 10:
    break

Index :  17
Correct answers :  ['Viking', 'Norseman, Viking', 'Norseman, Viking']
Predictions :  Norseman, Viking"

Index :  22
Correct answers :  ['King Charles III', 'King Charles III', 'King Charles III']
Predictions :  King Charles III of West Francia

Index :  23
Correct answers :  ['Seine', 'Epte', 'Seine']
Predictions :  river Seine

Index :  31
Correct answers :  ['Catholicism', 'Catholicism', 'Catholicism']
Predictions :  Norse

Index :  40
Correct answers :  ['Seljuk Turks', 'the Pechenegs, the Bulgars, and especially the Seljuk Turks', 'the Seljuk Turks']
Predictions :  the Pechenegs

Index :  47
Correct answers :  ['Alexius Komnenos', 'Alexius Komnenos', 'Alexius Komnenos']
Predictions :  Byzantine general Alexius Komnenos

Index :  54
Correct answers :  ['Turkish forces', 'Turkish forces', 'Turkish forces']
Predictions :  the Armenian state

Index :  59
Correct answers :  ['Norman mercenary', 'an Italo-Norman named Raoul', 'descended from an Italo-Norman named Raoul']
Pred

## 2 Incorrects predictions

In [ ]:
print("Correct answers : ", datasets['validation'][31])
print("Predictions : ", formatted_predictions[31]['prediction_text'])

Correct answers :  {'id': '56dde27d9a695914005b9651', 'title': 'Normans', 'context': 'The descendants of Rollo\'s Vikings and their Frankish wives would replace the Norse religion and Old Norse language with Catholicism (Christianity) and the Gallo-Romance language of the local people, blending their maternal Frankish heritage with Old Norse traditions and customs to synthesize a unique "Norman" culture in the north of France. The Norman language was forged by the adoption of the indigenous langue d\'oïl branch of Romance by a Norse-speaking ruling class, and it developed into the regional language that survives today.', 'question': 'What was the Norman religion?', 'answers': {'text': ['Catholicism', 'Catholicism', 'Catholicism'], 'answer_start': [121, 121, 121]}}
Predictions :  Norse


In [ ]:
print("Correct answers : ", datasets['validation'][54])
print("Predictions : ", formatted_predictions[54]['prediction_text'])

Correct answers :  {'id': '56de10b44396321400ee2595', 'title': 'Normans', 'context': 'Some Normans joined Turkish forces to aid in the destruction of the Armenians vassal-states of Sassoun and Taron in far eastern Anatolia. Later, many took up service with the Armenian state further south in Cilicia and the Taurus Mountains. A Norman named Oursel led a force of "Franks" into the upper Euphrates valley in northern Syria. From 1073 to 1074, 8,000 of the 20,000 troops of the Armenian general Philaretus Brachamius were Normans—formerly of Oursel—led by Raimbaud. They even lent their ethnicity to the name of their castle: Afranji, meaning "Franks." The known trade between Amalfi and Antioch and between Bari and Tarsus may be related to the presence of Italo-Normans in those cities while Amalfi and Bari were under Norman rule in Italy.', 'question': 'Who did the Normans team up with in Anatolia?', 'answers': {'text': ['Turkish forces', 'Turkish forces', 'Turkish forces'], 'answer_start': [20